In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 446 kB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


In [3]:
import numpy as np
import pandas as pd
import time
import datetime
import warnings
import random
import os
import yfinance as yf

warnings.filterwarnings(action='ignore')
%cd '/content/drive/MyDrive/Paper/Writing/Deep learning approach of the US stock market analysis and its explanation using correlation networks/code/'

/content/drive/MyDrive/Paper/Writing/Deep learning approach of the US stock market analysis and its explanation using correlation networks/code


In [18]:
def set_seed(seed_num):
  # tf.random.set_seed(seed_num)
  np.random.seed(seed_num)
  random.seed(seed_num)
  os.environ['PYTHONHASHSEED']=str(seed_num)

seed_num = 42
set_seed(seed_num)

- daily return -> z score normalization
- all anomalies

In [19]:
stock_id2 = ['MMM', 'AXP', 'AAPL', 'BA', 'CAT', 'CVX', 'CSCO', 'KO',
'DIS', 'XOM', 'GS', 'HD', 'IBM', 'INTC', 'JNJ', 'JPM', 'MCD', 'MRK', 'MSFT', 'NKE', 'PFE',
'PG', 'TRV', 'UNH', 'VZ', 'V', 'WMT', 'WBA', 'RTX']

In [20]:
all_orig_data = pd.DataFrame()
for st in stock_id2:
  print(st)
  data = yf.download(st, start='2019-12-31', interval='1d',  end='2021-01-30',progress = False)[['Close']]
  all_orig_data[st] = data

MMM
AXP
AAPL
BA
CAT
CVX
CSCO
KO
DIS
XOM
GS
HD
IBM
INTC
JNJ
JPM
MCD
MRK
MSFT
NKE
PFE
PG
TRV
UNH
VZ
V
WMT
WBA
RTX


In [21]:
all_orig_data

,MMM,AXP,AAPL,BA,CAT,CVX,CSCO,KO,DIS,XOM,...,NKE,PFE,PG,TRV,UNH,VZ,V,WMT,WBA,RTX
Date,,,,,,,,,,,,,,,,,,,,,
2019-12-31,176.419998,124.489998,73.412498,325.760010,147.679993,120.510002,47.959999,55.349998,144.630005,69.779999,...,101.309998,37.172676,124.900002,136.949997,293.980011,61.400002,187.899994,118.839996,58.959999,94.247955
2020-01-02,180.000000,125.849998,75.087502,333.320007,150.529999,121.430000,48.419998,54.990002,148.199997,70.900002,...,102.199997,37.134724,123.410004,137.509995,292.500000,61.049999,191.119995,118.940002,59.080002,96.375076
2020-01-03,178.449997,124.599998,74.357498,332.760010,148.440002,121.010002,47.630001,54.689999,146.500000,70.330002,...,101.919998,36.935486,122.580002,137.020004,289.540009,60.400002,189.600006,117.889999,59.080002,96.507240
2020-01-06,178.619995,124.059998,74.949997,333.739990,148.339996,120.599998,47.799999,54.669998,145.649994,70.870003,...,101.830002,36.888046,122.750000,137.169998,291.549988,60.270000,189.190002,117.650002,59.590000,96.714912
2020-01-07,177.899994,123.410004,74.597504,337.279999,146.380005,119.059998,47.490002,54.250000,145.699997,70.290001,...,101.779999,36.764706,121.989998,135.160004,289.790009,59.599998,188.690002,116.559998,59.290001,96.406548
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-01-25,170.389999,121.150002,142.919998,203.360001,187.339996,90.900002,45.029999,48.779999,171.889999,46.900002,...,137.550003,37.279999,132.240005,146.360001,347.809998,58.419998,200.979996,146.199997,47.820000,66.279999
2021-01-26,175.949997,116.150002,143.160004,202.059998,187.210007,89.360001,45.259998,49.290001,169.559998,45.869999,...,135.100006,37.310001,133.089996,143.710007,343.109985,56.570000,202.009995,147.509995,49.189999,67.199997
2021-01-27,186.649994,114.089996,142.059998,194.029999,180.630005,88.199997,45.700001,48.529999,163.029999,45.349998,...,131.020004,36.240002,128.380005,139.500000,332.989990,55.130001,194.970001,143.839996,51.180000,65.500000


In [22]:
daily_return = all_orig_data.pct_change()

In [23]:
daily_return.isnull().sum()

MMM     1
AXP     1
AAPL    1
BA      1
CAT     1
CVX     1
CSCO    1
KO      1
DIS     1
XOM     1
GS      1
HD      1
IBM     1
INTC    1
JNJ     1
JPM     1
MCD     1
MRK     1
MSFT    1
NKE     1
PFE     1
PG      1
TRV     1
UNH     1
VZ      1
V       1
WMT     1
WBA     1
RTX     1
dtype: int64

In [24]:
daily_return = daily_return.dropna()

In [25]:
daily_return

,MMM,AXP,AAPL,BA,CAT,CVX,CSCO,KO,DIS,XOM,...,NKE,PFE,PG,TRV,UNH,VZ,V,WMT,WBA,RTX
Date,,,,,,,,,,,,,,,,,,,,,
2020-01-02,0.020292,0.010925,0.022816,0.023207,0.019299,0.007634,0.009591,-0.006504,0.024684,0.016050,...,0.008785,-0.001021,-0.011930,0.004089,-0.005034,-0.005700,0.017137,0.000842,0.002035,0.022569
2020-01-03,-0.008611,-0.009932,-0.009722,-0.001680,-0.013884,-0.003459,-0.016316,-0.005456,-0.011471,-0.008039,...,-0.002740,-0.005365,-0.006726,-0.003563,-0.010120,-0.010647,-0.007953,-0.008828,0.000000,0.001371
2020-01-06,0.000953,-0.004334,0.007968,0.002945,-0.000674,-0.003388,0.003569,-0.000366,-0.005802,0.007678,...,-0.000883,-0.001284,0.001387,0.001095,0.006942,-0.002152,-0.002162,-0.002036,0.008632,0.002152
2020-01-07,-0.004031,-0.005239,-0.004703,0.010607,-0.013213,-0.012769,-0.006485,-0.007682,0.000343,-0.008184,...,-0.000491,-0.003344,-0.006191,-0.014653,-0.006037,-0.011117,-0.002643,-0.009265,-0.005034,-0.003188
2020-01-08,0.015346,0.017260,0.016086,-0.017523,0.008881,-0.011423,0.000632,0.001843,-0.002059,-0.015080,...,-0.002260,0.008000,0.004263,0.010728,0.021084,0.001846,0.017118,-0.003432,-0.058357,0.001501
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-01-25,0.007986,-0.039559,0.027684,-0.012048,-0.023966,-0.009048,0.005807,0.005981,-0.005151,-0.011174,...,-0.012917,0.019973,0.017231,0.003566,0.000748,0.016530,-0.005148,-0.000888,0.007798,-0.019237
2021-01-26,0.032631,-0.041271,0.001679,-0.006393,-0.000694,-0.016942,0.005108,0.010455,-0.013555,-0.021962,...,-0.017812,0.000805,0.006428,-0.018106,-0.013513,-0.031667,0.005125,0.008960,0.028649,0.013880
2021-01-27,0.060813,-0.017736,-0.007684,-0.039741,-0.035148,-0.012981,0.009722,-0.015419,-0.038511,-0.011336,...,-0.030200,-0.028679,-0.035390,-0.029295,-0.029495,-0.025455,-0.034850,-0.024880,0.040455,-0.025298


In [26]:
daily_return.mean(axis=1)

Date
2020-01-02    0.010033
2020-01-03   -0.008067
2020-01-06    0.001608
2020-01-07   -0.006016
2020-01-08    0.002798
                ...   
2021-01-25   -0.001573
2021-01-26   -0.000652
2021-01-27   -0.018634
2021-01-28    0.011176
2021-01-29   -0.018839
Length: 272, dtype: float64

In [27]:
daily_return.std(axis=1)

Date
2020-01-02    0.010137
2020-01-03    0.004809
2020-01-06    0.004521
2020-01-07    0.007730
2020-01-08    0.015456
                ...   
2021-01-25    0.015288
2021-01-26    0.018123
2021-01-27    0.023296
2021-01-28    0.020102
2021-01-29    0.011741
Length: 272, dtype: float64

In [28]:
mat1 = daily_return.sub(daily_return.mean(axis=1), axis=0)
zscore = mat1.div(daily_return.std(axis=1), axis=0)

In [29]:
zscore

,MMM,AXP,AAPL,BA,CAT,CVX,CSCO,KO,DIS,XOM,...,NKE,PFE,PG,TRV,UNH,VZ,V,WMT,WBA,RTX
Date,,,,,,,,,,,,,,,,,,,,,
2020-01-02,1.012055,0.087942,1.261022,1.299587,0.914003,-0.236642,-0.043580,-1.631328,1.445224,0.593595,...,-0.123129,-1.090446,-2.166537,-0.586358,-1.486355,-1.552051,0.700755,-0.906717,-0.788952,1.236665
2020-01-03,-0.113206,-0.387975,-0.344218,1.328100,-1.209745,0.958220,-1.715322,0.542983,-0.707905,0.005666,...,1.107746,0.561765,0.278894,0.936481,-0.426897,-0.536557,0.023638,-0.158306,1.677466,1.962636
2020-01-06,-0.144901,-1.314206,1.406856,0.295784,-0.504628,-1.105031,0.433834,-0.436501,-1.638955,1.342680,...,-0.550921,-0.639701,-0.048863,-0.113482,1.179859,-0.831679,-0.833920,-0.805898,1.553744,0.120355
2020-01-07,0.256752,0.100425,0.169803,2.150364,-0.931046,-0.873695,-0.060755,-0.215618,0.822613,-0.280506,...,0.714679,0.345659,-0.022743,-1.117391,-0.002713,-0.659882,0.436316,-0.420318,0.126940,0.365743
2020-01-08,0.811859,0.935680,0.859771,-1.314729,0.393581,-0.920075,-0.140146,-0.061755,-0.314237,-1.156718,...,-0.327221,0.336585,0.094781,0.513089,1.183136,-0.061602,0.926523,-0.403045,-3.956743,-0.083875
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-01-25,0.625288,-2.484616,1.913682,-0.685148,-1.464671,-0.488930,0.482769,0.494093,-0.234014,-0.627991,...,-0.741986,1.409300,1.229961,0.336132,0.151845,1.184142,-0.233818,0.044799,0.612948,-1.155329
2021-01-26,1.836477,-2.241321,0.128614,-0.316781,-0.002334,-0.898862,0.317787,0.612850,-0.712000,-1.175856,...,-0.946866,0.080360,0.390619,-0.963106,-0.709681,-1.711391,0.318735,0.530368,1.616761,0.801854
2021-01-27,3.410308,0.038564,0.470052,-0.906012,-0.708856,0.242654,1.217191,0.138012,-0.853247,0.313260,...,-0.496467,-0.431167,-0.719236,-0.457632,-0.466206,-0.292798,-0.696065,-0.268094,2.536457,-0.286033


In [30]:
zscore.to_csv('return_zscore/all_zscore.csv')